In [5]:
import sqlite3
import xml.etree.ElementTree as ET
from datetime import datetime

Creating XML file that fits OPML structure (see [OPML Spec](https://opml.org/spec2.opml))

In [7]:
def create_opml_structure():
    """Create the basic OPML structure."""
    opml = ET.Element("opml", version="2.0")
    head = ET.SubElement(opml, "head")
    ET.SubElement(head, "title").text = "BeyondPod Feeds Export"
    ET.SubElement(head, "dateCreated").text = datetime.now().strftime("%a, %d %b %Y %H:%M:%S %z")
    body = ET.SubElement(opml, "body")    
    return opml, body

Finding out the structure of tables in SQlite DB:

```
pragma table_info('feeds')
```


0|_id|INTEGER|0||1
1|feedId|TEXT|0||0
2|path|TEXT|0||0
3|name|TEXT|0||0
4|autodelete|INTEGER|0||0
5|view|INTEGER|0||0
6|type|INTEGER|0||0
7|url|TEXT|0||0
8|imageUrl|TEXT|0||0
9|hasUnread|INTEGER|0||0
10|pubDate|INTEGER|0||0
11|srvPubDate|INTEGER|0||0
12|category|TEXT|0||0
13|custDownload|INTEGER|0||0
14|forceUniqueNames|INTEGER|0||0
15|leftTruncateLongTrackNames|INTEGER|0||0
16|forceItemSort|INTEGER|0||0
17|lastItemId|TEXT|0||0
18|trackSort|INTEGER|0||0
19|feedPlayer|INTEGER|0||0
20|username|TEXT|0||0
21|maxDownload|INTEGER|0||0
22|maxGReaderItems|INTEGER|0||0
23|maxTracks|INTEGER|0||0
24|maxTrackAge|INTEGER|0||0
25|savePlayedPosition|INTEGER|0||0
26|fingerprintType|INTEGER|0||0
27|audioSettings|TEXT|0||0

In [16]:
cursor = connect_to_db("beyondpod.db").cursor()
cursor.execute("SELECT * FROM feeds")
cursor.fetchall()

[(1,
  'be940479-4148-443e-a7ac-112431cf7456',
  '/storage/emulated/0/Android/data/mobi.beyondpod/files/BeyondPod/Podcasts/BeyondPod Quick Start',
  'BeyondPod Quick Start',
  2,
  0,
  3,
  'http://www.beyondpod.com/cms/welcome',
  'http://www.beyondpod.com/external/BeyondPodQS.jpg',
  7,
  1317135600000,
  1373654777000,
  'Old|Uncategorized',
  0,
  1,
  0,
  -1,
  None,
  None,
  -1,
  '',
  1,
  0,
  5,
  99999,
  1,
  1,
  '0|'),
 (8,
  '44ee292f-36c7-48d6-b534-3cad7ea6da02',
  '/storage/emulated/0/Android/data/mobi.beyondpod/files/BeyondPod/Podcasts/BeyondPod Blog',
  'BeyondPod Blog',
  2,
  0,
  3,
  'http://feeds.feedburner.com/beyondpodforandroid',
  'http://www.beyondpod.com/external/BeyondPod.png',
  8,
  1327962686000,
  1328364004000,
  'Old|Uncategorized',
  0,
  1,
  0,
  -1,
  None,
  None,
  -1,
  '',
  0,
  0,
  999,
  99999,
  1,
  1,
  '0|'),
 (10,
  '5e6826f7-0147-4765-bbc1-5d0ad7344e62',
  '/storage/emulated/0/Android/data/mobi.beyondpod/files/BeyondPod/Podcasts

In [26]:
conn = sqlite3.connect('beyondpod.db')
cursor = conn.cursor()
# Query to get feed information
cursor.execute("""
    SELECT name, url
    FROM feeds
""")

# Create OPML structure
opml, body = create_opml_structure()

# Get feeds and add them to OPML
for feed in cursor.fetchall():
    name, url = feed
    outline = ET.SubElement(body, "outline")
    outline.set("type", "rss")
    outline.set("text", name or "")
    outline.set("title", name or "")
    outline.set("xmlUrl", url or "")

# Write to file
tree = ET.ElementTree(opml)
tree.write('beyondpod.opml', encoding="utf-8", xml_declaration=True)
conn.close()